In [ ]:
export REGION=us-central1
export ZONE=us-central1-a
export MonolithIdentifier=fancy-monolith-164
export ClusterName=fancy-prod-837
export OrdersIdentifier=fancy-orders-528
export ProductsIdentifier=fancy-products-530
export FrontendIdentifier=fancy-frontend-647

gcloud config set compute/region $REGION
gcloud config set compute/zone $ZONE

# Task 1: Build a container

git clone https://github.com/googlecodelabs/monolith-to-microservices.git
    
cd ~/monolith-to-microservices
./setup.sh

nvm install --lts

cd ~/monolith-to-microservices/monolith
# npm start

gcloud services enable cloudbuild.googleapis.com
gcloud builds submit . \
    --tag gcr.io/${GOOGLE_CLOUD_PROJECT}/$MonolithIdentifier:1.0.0
    
# Task 2: Create a kubernetes & deploy the application

gcloud services enable container.googleapis.com
gcloud container clusters create $ClusterName \
    --zone=$ZONE \
    --machine-type=n1-standard-1 \
    --num-nodes=3

kubectl create deployment $MonolithIdentifier \
    --image=gcr.io/${GOOGLE_CLOUD_PROJECT}/$MonolithIdentifier:1.0.0

kubectl expose deployment $MonolithIdentifier \
    --type=LoadBalancer --port=80 --target-port=8080

# Task 3: Create a microservice

cd ~/monolith-to-microservices/microservices/src/orders
gcloud builds submit . \
    --tag gcr.io/${GOOGLE_CLOUD_PROJECT}/$OrdersIdentifier:1.0.0

cd ~/monolith-to-microservices/microservices/src/products
gcloud builds submit . \
    --tag gcr.io/${GOOGLE_CLOUD_PROJECT}/$ProductsIdentifier:1.0.0
    
# Task 4: Deploy the microservice

kubectl create deployment $OrdersIdentifier \
    --image=gcr.io/${GOOGLE_CLOUD_PROJECT}/$OrdersIdentifier:1.0.0
kubectl expose deployment $OrdersIdentifier \
    --type=LoadBalancer --port=80 --target-port=8081

kubectl create deployment $ProductsIdentifier \
    --image=gcr.io/${GOOGLE_CLOUD_PROJECT}/$ProductsIdentifier:1.0.0
kubectl expose deployment $ProductsIdentifier \
    --type=LoadBalancer --port=80 --target-port=8082

# Task 5: Configure the frontend microservice

kubectl get svc $OrdersIdentifier
kubectl get svc $ProductsIdentifier

cd ~/monolith-to-microservices/react-app
nano .env

REACT_APP_ORDERS_URL=http://104.197.196.15/api/orders
REACT_APP_PRODUCTS_URL=http://34.27.74.73/api/products

npm run build

# Task 6: Create the frontend microservice

cd ~/monolith-to-microservices/microservices/src/frontend
gcloud builds submit . \
    --tag gcr.io/${GOOGLE_CLOUD_PROJECT}/$FrontendIdentifier:1.0.0
    
# Task 7: Deploy the frontend microservice

kubectl create deployment $FrontendIdentifier \
    --image=gcr.io/${GOOGLE_CLOUD_PROJECT}/$FrontendIdentifier:1.0.0

kubectl expose deployment $FrontendIdentifier \
    --type=LoadBalancer --port=80 --target-port=8080